In [1]:
import glob
import math
import sys
import timeit

sys.path.insert(0, '../../')
from visualize_dataset import *
from losses import *
from metrics import *
from predict import *
from train import *
from utils import *

Using TensorFlow backend.


In [2]:
path_train_images = '../../../data/stanford_max_lesion_area_fused_cropped/train/Slice/'
path_train_annotations = '../../../data/stanford_max_lesion_area_fused_cropped/train/ROI/'
path_train_predictions = './predictions.50e/train/'

path_val_images = '../../../data/stanford_max_lesion_area_fused_cropped/val/Slice/'
path_val_annotations = '../../../data/stanford_max_lesion_area_fused_cropped/val/ROI/'
path_val_predictions = './predictions.50e/val/'

path_test_images = '../../../data/stanford_max_lesion_area_fused_cropped/test/Slice/'
path_test_annotations = '../../../data/stanford_max_lesion_area_fused_cropped/test/ROI/'
path_test_predictions = './predictions.50e/test/'

path_save_weights = './weights.50e/'
path_load_weights = None

train_vgg = False
path_vgg_weights = '../../../data/vgg16_weights_th_dim_ordering_th_kernels.h5'

n_classes = 2
input_height = 224
input_width = 224
n_channel_annotations = 1

In [3]:
print('Number train images: ', len(glob.glob(path_train_images + '*.png')))
print('Number val images: ', len(glob.glob(path_val_images + '*.png')))
print('Number test images: ', len(glob.glob(path_test_images + '*.png')))

('Number train images: ', 376)
('Number val images: ', 15)
('Number test images: ', 14)


In [4]:
model_name = 'vgg_segnet'
epochs = 50
train_batch_size = 8
val_batch_size = len(glob.glob(path_val_images + '*.png'))

In [5]:
optimizer = 'adadelta'
metrics = ['accuracy', dice, iou]

class_props = calc_class_proportions(path_train_annotations, n_classes, n_channel_annotations)
class_weights = calc_class_weights(path_train_annotations, n_classes, n_channel_annotations)
class_weights_log = calc_class_weights(path_train_annotations, n_classes, n_channel_annotations, scale = 'log')

print(class_props)
print(class_weights)
print(class_weights_log)

loss = weighted_categorical_crossentropy_loss(class_weights_log)

[0.97803434 0.02196566]
[ 1.         44.52560761]
[0.0222105  3.81827497]


In [ ]:
start = timeit.default_timer()
train(
    path_train_images, path_train_annotations, train_batch_size, 
    path_val_images, path_val_annotations, val_batch_size,
    n_classes, input_width, input_height, 
    model_name, epochs, loss, optimizer, metrics,
    path_save_weights, path_vgg_weights = path_vgg_weights, train_vgg = train_vgg, path_load_weights = None
)
end = timeit.default_timer()

('Model output shape', (None, 50176, 2))
('Epoch', 0)
Epoch 1/1
47/47 [==============================] - 1196s 25s/step - loss: 0.0354 - acc: 0.5768 - dice: 0.0814 - iou: 0.0426 - val_loss: 0.0311 - val_acc: 0.5474 - val_dice: 0.0695 - val_iou: 0.0360
('Epoch', 1)
Epoch 1/1
47/47 [==============================] - 1217s 26s/step - loss: 0.0235 - acc: 0.6484 - dice: 0.1022 - iou: 0.0541 - val_loss: 0.0329 - val_acc: 0.6397 - val_dice: 0.0828 - val_iou: 0.0432
('Epoch', 2)
Epoch 1/1
47/47 [==============================] - 1198s 25s/step - loss: 0.0190 - acc: 0.7569 - dice: 0.1409 - iou: 0.0762 - val_loss: 0.0348 - val_acc: 0.6778 - val_dice: 0.0895 - val_iou: 0.0469
('Epoch', 3)
Epoch 1/1
47/47 [==============================] - 1227s 26s/step - loss: 0.0158 - acc: 0.8448 - dice: 0.2060 - iou: 0.1159 - val_loss: 0.0380 - val_acc: 0.7529 - val_dice: 0.1059 - val_iou: 0.0559
('Epoch', 4)
Epoch 1/1
47/47 [==============================] - 1193s 25s/step - loss: 0.0139 - acc: 0.8881 - dice:

In [ ]:
print('Total training time: ', end - start)

In [6]:
epochs = 42

In [7]:
predict(
    path_train_images, path_train_annotations, path_train_predictions,
    n_classes, input_width, input_height, 
    model_name, epochs, loss, optimizer, metrics, 
    path_save_weights
)

              precision    recall  f1-score   support

           0       1.00      0.99      1.00  18491024
           1       0.74      0.99      0.85    375152

   micro avg       0.99      0.99      0.99  18866176
   macro avg       0.87      0.99      0.92  18866176
weighted avg       0.99      0.99      0.99  18866176

('F1/dice (binary): ', 0.8473716078478054)
('Jaccard/IOU: ', 0.7351646147337982)


In [ ]:
visualize_dataset(
    path_train_images, path_train_annotations, 
    n_classes, n_channel_annotations = n_channel_annotations,
    dir_predictions = path_train_predictions
)

In [9]:
predict(
    path_val_images, path_val_annotations, path_val_predictions,
    n_classes, input_width, input_height, 
    model_name, epochs, loss, optimizer, metrics,
    path_save_weights
)

              precision    recall  f1-score   support

           0       0.98      0.99      0.99    739775
           1       0.13      0.11      0.12     12865

   micro avg       0.97      0.97      0.97    752640
   macro avg       0.56      0.55      0.55    752640
weighted avg       0.97      0.97      0.97    752640

('F1/dice (binary): ', 0.12196369215034518)
('Jaccard/IOU: ', 0.06494213750850919)


In [ ]:
visualize_dataset(
    path_val_images, path_val_annotations, 
    n_classes, n_channel_annotations = n_channel_annotations,
    dir_predictions = path_val_predictions
)

In [11]:
predict(
    path_test_images, path_test_annotations, path_test_predictions,
    n_classes, input_width, input_height, 
    model_name, epochs, loss, optimizer, metrics,
    path_save_weights
)

              precision    recall  f1-score   support

           0       0.98      0.99      0.99    686571
           1       0.38      0.23      0.29     15893

   micro avg       0.97      0.97      0.97    702464
   macro avg       0.68      0.61      0.64    702464
weighted avg       0.97      0.97      0.97    702464

('F1/dice (binary): ', 0.28548877224831287)
('Jaccard/IOU: ', 0.16651321241138017)


In [ ]:
visualize_dataset(
    path_test_images, path_test_annotations, 
    n_classes, n_channel_annotations = n_channel_annotations, 
    dir_predictions = path_test_predictions
)